In [48]:
"""
Load a BehaviourStore setup json and interactively edit it.
"""
import os
#fom BluenetTestSuite import ...
from ipywidgets import Layout, VBox, HBox, Accordion, Tab
from ipywidgets import IntRangeSlider, FloatText, IntSlider, BoundedIntText
from ipywidgets import Button, ToggleButton, ToggleButtons
from ipywidgets import Checkbox, Select
from ipywidgets import Text, Label, Output

In [16]:
icon_expand = 'bars'
icon_ellipsis = 'ellipsis-h'
icon_pencil = 'pencil'
icon_cog = 'cog'

icon_edit = icon_cog
icon_collapse = 'minus' #'window-minimize'
icon_create = 'plus'
icon_save = 'fa-floppy-o'
icon_delete = 'fa-trash-o'
icon_reload = 'fa-undo'

In [37]:
def MakeHBox(widgetlistlist, widgetwidthlist):
    hbox_layout = Layout(
        overflow='scroll',
        height='',
        margin='5px 5px 0 0',
        flex_flow='row',
        display='stretch'
    )
    
    marge = '5px'
    
    columns = [
        VBox(children = widgetlistlist[i], 
             layout = Layout(width = widgetwidthlist[i], margin=marge)
            ) for i in range(min(len(widgetlistlist), len(widgetwidthlist)))
    ]
    
    return HBox(children = columns, layout = hbox_layout)

def MakeHBox_single(widgetlist, widgetwidthlist):
    return MakeHBox([[widg] for widg in widgetlist], widgetwidthlist)


def MetaDataSummary():
    indexfield = BoundedIntText(
        value=0,
        min=0,
        max=49,
        step=1,
        tooltip="index in behaviourstore",
        disabled=False,
        layout=Layout(width='100%')
    )
    
    return [indexfield]

def MetaDataDetails():
    return []

def BehaviourOverviewSummary():
    """
    Returns list of widgets for single line description of a behaviour,
    and a callback taking in a behaviour description dict to update its value.
    """
    color_dict = {
        'Switch': "lightgreen",
        'ExtendedSwitch': "lightblue", 
        'Twilight': "#DADADA"
    }
    
    no_color = "#FFFFFF"
        
    summarywidget_left = Button(description="", layout=Layout(width='25%'))
    summarywidget_middle = Button(description="uninitialized", layout=Layout(width='50%'))
    summarywidget_right = Button(description="", layout=Layout(width='25%'))
    
    summarywidget_left.style.button_color = no_color
    summarywidget_middle.style.button_color = no_color
    summarywidget_right.style.button_color = no_color
    
    summarywidget = HBox([summarywidget_left, summarywidget_middle, summarywidget_right])
    
    def update_summary(behaviour_settings_dict):
        relative_min = int(100*behaviour_settings_dict['from']/float(24*60*60))
        relative_max = int(100*behaviour_settings_dict['until']/float(24*60*60))
        summarywidget_left.layout.width = str(relative_min) + "%"
        summarywidget_middle.layout.width = str(relative_max - relative_min) + "%"
        summarywidget_right.layout.width = str(100 - relative_max) + "%"
        
        summarywidget_middle.description = "{0} ({1}%)".format(
            behaviour_settings_dict['type'],
            behaviour_settings_dict['intensity']
        )
        
        active_color = color_dict[behaviour_settings_dict['type']]
        not_reversed = not behaviour_settings_dict['fromuntil_reversed']
        summarywidget_left.style.button_color = no_color if not_reversed else active_color
        summarywidget_middle.style.button_color = active_color if not_reversed else no_color 
        summarywidget_right.style.button_color = no_color if not_reversed else active_color
        
    
    return [summarywidget], update_summary

def BehaviourOverviewDetails():
    """
    Returns list of widgets for multi line description of a behaviour
    """
    fromuntilfield = IntRangeSlider(
        value=[9*60*60, 18*60*60],
        min=0,
        max=24*60*60,
        step=60,
        disabled=False,
        orientation='horizontal',
        readout=False,
        readout_format='.1f',
        layout=Layout(width='100%')
    )
    
    fromfield = BoundedIntText(
        value=9*60*60,
        min=0,
        max=24*60*60,
        description='From:',
        disabled=False,
    )
    
    untilfield = BoundedIntText(
        value=18*60*60,
        min=0,
        max=24*60*60,
        description='Until:',
        disabled=False,
    )
    
    def on_range_field_change(change):
        fromval,untilval = change['new']
        fromfield.value = fromval
        untilfield.value = untilval        
        
    def on_from_field_change(change):
        prev_from_until = fromuntilfield.value
        next_from_until = (change['new'], prev_from_until[1])
        
        if next_from_until[0] <= next_from_until[1]:
            fromuntilfield.value = next_from_until
    
    def on_until_field_change(change):
        prev_from_until = fromuntilfield.value
        next_from_until = (prev_from_until[0], change['new'])
        
        if next_from_until[0] <= next_from_until[1]:
            fromuntilfield.value = next_from_until
        
    fromuntilfield.observe(on_range_field_change, names='value')
    fromfield.observe(on_from_field_change, names='value')
    untilfield.observe(on_until_field_change, names='value')
    
    intensityfield = IntSlider(
        value=100,
        min=0,
        max=100,
        description='Intensity:',
        disabled=False,
        orientation='horizontal',
    )
    
    # only necessary because the range slider doesn't include reversed ranges...
    fromuntil_reversed_field = Checkbox(
        value=False,
        description='Midnight:',
        tooltip='Reverses from/until times when checked',
        disabled=False,
        indent=False
    )
    
    typefield = ToggleButtons(
        options=['Switch', 'ExtendedSwitch', 'Twilight'],
        description='Type:',
        disabled=False,
        button_style='info',
        layout=Layout(width='100%')
    )
    
    def get_behaviour_settings_dict():
            settings = dict()
            settings['intensity'] = intensityfield.value
            settings['type'] = typefield.value
            settings['fromuntil_reversed'] = fromuntil_reversed_field.value
            settings['from'] = fromfield.value
            settings['until'] = untilfield.value
            return settings
    
    return [fromuntilfield, fromfield, untilfield, 
            intensityfield, fromuntil_reversed_field, typefield], get_behaviour_settings_dict

def ToolbarSummary():
    editbutton = ToggleButton(
        value = False,  # meaning 'collapsed'
        tooltip='Expand',
        disabled=False,
        icon=icon_edit,
        layout=Layout(width='100%')
    )
    
    return [editbutton]
    
def ToolbarDetails():
    deletebutton = Button(
        tooltip='Delete',
        disabled=False,
        icon=icon_delete,
        layout=Layout(width='100%')
    )
    
    savebutton = Button(
        tooltip='Save',
        disabled=False,
        icon=icon_save,
        layout=Layout(width='100%')
    )
    
    reloadbutton = Button(
        tooltip='Reload',
        disabled=False,
        icon=icon_reload,
        layout=Layout(width='100%')
    )
    
    return [savebutton, reloadbutton, deletebutton]

            
def BehaviourEntryEditor():
    """
    Returns HBox for Behaviour with additional meta-operations
    """
    summary_meta = MetaDataSummary()
    summary_overview, overview_update_callback = BehaviourOverviewSummary()
    summary_toolbar = ToolbarSummary()
    
    details_meta = MetaDataDetails()
    details_overview, overview_get_behaviour_settings_dict = BehaviourOverviewDetails()
    details_toolbar = ToolbarDetails()
    
    summary = MakeHBox([summary_meta, summary_overview, summary_toolbar], ['5%', '90%', '5%'])
    details = MakeHBox([details_meta, details_overview, details_toolbar], ['5%', '90%', '5%'])

    entry_editor = VBox([summary])
    
    editbutton = summary_toolbar[0]
    
    def toggle_detail_widgets(observation):
        if editbutton.value:
            entry_editor.children = [summary, details]
            editbutton.tooltip = "Collapse"     
            editbutton.icon = icon_collapse
        else:
            entry_editor.children = [summary]
            editbutton.tooltip = "Expand"
            editbutton.icon = icon_edit
    
    def update_summary_widget(observation):
        overview_update_callback(
            overview_get_behaviour_settings_dict()
        )
        
    editbutton.observe(toggle_detail_widgets, 'value')
    
    for behaviour_edit_widget in details_overview:
        behaviour_edit_widget.observe(update_summary_widget, 'value')
    
    return entry_editor

In [38]:
VBox([BehaviourEntryEditor() for i in range(3)])

In [60]:
file_ext = ".behaviourstore.json"

output_field = Output()

behaviour_store_folder_field = Text(    
    value=os.getcwd(),
    placeholder=os.getcwd(),
    description='Folder:',
    layout=Layout(width='100%')
)

loadbutton = Button(
    tooltip='Load store files',
    disabled=False,
    icon='upload',
    layout=Layout(width='100%')
)

fileselector = Select(
    options=[],
    description='Read file:',
    disabled=True
)

reloadbutton = Button(
    tooltip='Reload from file',
    disabled=False,
    icon=icon_reload,
    layout=Layout(width='100%')
)

deletebutton = Button(
    tooltip='Delete store file',
    disabled=False,
    icon=icon_delete,
    layout=Layout(width='100%')
)

filename_for_saving_field = Text(    
    value="new_file_name",
    placeholder="file_name",
    description='Write file:',
    layout=Layout(width='100%')
)

createbutton = Button(
    tooltip='Create store file',
    disabled=False,
    icon=icon_create,
    layout=Layout(width='100%')
)

savebutton = Button(
    tooltip='Save to file',
    disabled=False,
    icon=icon_save,
    layout=Layout(width='100%')
)


def reload_file_selector(button):
    # load file selector and update current working folder
    try:
        fileselector.options = [fil[:-len(".behaviourstore.json")] for fil in os.listdir(behaviour_store_folder_field.value) if
                                fil.endswith(file_ext)]
    except:
        fileselector.disabled=True
        with output_field:
            print("somethng bad happened")
    finally:
        fileselector.disabled=False

loadbutton.on_click(reload_file_selector)


def file_path_invalid():
    # check if behaviour_store_folder_field.value is ok
    return False
        

def create_button_click(button):
    if file_path_invalid():
        with output_field:
            print("path invalid!")
        return
    with open(behaviour_store_folder_field.value + "/" +
                 filename_for_saving_field.value +
                  file_ext, 'w'):
        pass
    reload_file_selector(None)
    fileselector.value=filename_for_saving_field.value
    
createbutton.on_click(save_button_click)
    
def delete_button_click(button):
    if file_path_invalid():
        with output_field:
            print("file path invalid!")
        return
    try:
        os.remove(behaviour_store_folder_field.value + "/" +
                 fileselector.value +
                  file_ext
                 )
    except:
        pass
    reload_file_selector(None)
    
deletebutton.on_click(delete_button_click)


VBox(children=[
    MakeHBox_single([behaviour_store_folder_field, loadbutton], ['95%','5%']),
    MakeHBox_single([fileselector, reloadbutton, deletebutton], 
         ['90%', '5%', '5%']),
    MakeHBox_single([filename_for_saving_field, savebutton, createbutton],
                   ['90%', '5%', '5%']),
    output_field
    ]
)


